In [6]:
from tensorflow.keras.utils import normalize
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2
import matplotlib.pyplot as plt
from glob import glob
import os
import random
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

input_dir = "../../../../Desktop/Ateroskleroza_Data_Original_26_9_19/images/"
mask_dir = "../../../../Desktop/Ateroskleroza_Data_Original_26_9_19/labels/"
model = load_model('./Third_training/20230325_atherosceloris_segmentation.h5')

In [8]:
n_classes = 4

train_images = []
for directory_path in sorted(os.listdir(input_dir)):
    input_path = input_dir + directory_path
    img = cv2.imread(input_path, 0)
    train_images.append(img)       
train_images = np.array(train_images)
print(train_images.shape)

train_masks = []
for directory_path in sorted(os.listdir(mask_dir)):
    input_path = mask_dir + directory_path
    mask = cv2.imread(input_path, 0)
    train_masks.append(mask)
train_masks = np.array(train_masks)
print(train_masks.shape)

labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1, 1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_reshaped_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_reshaped_encoded_original_shape)

train_images = np.expand_dims(train_images, axis=3)
train_images = normalize(train_images, axis=1)

train_masks_input = np.expand_dims(train_masks, axis=3)

X_train, X_test, y_train, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10, random_state=42)

train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))

test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

(869, 544, 544)
(869, 544, 544)


In [9]:
batch_size = 16
seed = 100

img_data_gen_args = dict(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='reflect')

mask_data_gen_args = dict(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='reflect',
    )

image_data_generator = ImageDataGenerator(**img_data_gen_args)
mask_data_generator = ImageDataGenerator(**mask_data_gen_args)

image_data_generator.fit(X_train, augment=True, seed=seed)
image_generator = image_data_generator.flow(X_train, seed=seed)
valid_img_generator = image_data_generator.flow(X_test, seed=seed)


mask_data_generator.fit(y_train_cat, augment=True, seed=seed)
mask_generator = mask_data_generator.flow(y_train_cat, seed=seed)
valid_mask_generator = mask_data_generator.flow(y_test_cat, seed=seed)

train_generator = zip(image_generator, mask_generator)
val_generator = zip(valid_img_generator, valid_mask_generator)

In [10]:
train_loss, train_acc = model.evaluate_generator(train_generator, steps=16)
test_loss, test_acc = model.evaluate_generator(val_generator, steps=16)

/Users/davidpilny/miniforge3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
2023-03-28 14:53:23.057133: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-03-28 14:53:23.058358: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [16]:
print('Training accruracy:\t{0}\nTraining loss:\t\t{1}\nTesting accuracy:\t{2}\nTesting loss:\t\t{3}'.format(train_acc, train_loss, test_acc, test_loss))

Training accruracy:	0.9742642045021057
Training loss:		0.06470072269439697
Testing accuracy:	0.9711969494819641
Testing loss:		0.0739593505859375


In [17]:
model1 = load_model('../20221030_first_training/20221030_atherosclerosis_model.h5')
train_loss, train_acc = model1.evaluate_generator(train_generator, steps=16)
test_loss, test_acc = model1.evaluate_generator(val_generator, steps=16)
print('Training accruracy:\t{0}\nTraining loss:\t\t{1}\nTesting accuracy:\t{2}\nTesting loss:\t\t{3}'.format(train_acc, train_loss, test_acc, test_loss))

/Users/davidpilny/miniforge3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Training accruracy:	0.8838330507278442
Training loss:		0.39730024337768555
Testing accuracy:	0.8815974593162537
Testing loss:		0.40781015157699585


In [22]:
model2 = load_model('../20230326_fourth_training/Training_Output/20230326_atherosceloris_segmentation.h5')
train_loss, train_acc = model2.evaluate_generator(train_generator, steps=16)
test_loss, test_acc = model2.evaluate_generator(val_generator, steps=16)
print('Training accruracy:\t{0}\nTraining loss:\t\t{1}\nTesting accuracy:\t{2}\nTesting loss:\t\t{3}'.format(train_acc, train_loss, test_acc, test_loss))

Training accruracy:	0.9742568731307983
Training loss:		0.06514639407396317
Testing accuracy:	0.9726105332374573
Testing loss:		0.07123444974422455
